In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras import models, layers
from tensorflow.keras.utils import to_categorical
from sklearn.utils import class_weight

In [2]:
df = pd.read_csv("healthcare-dataset-stroke-data.csv")
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [3]:
df = df.drop(columns=["id"])
# Xử lý thiếu BMI
df['bmi'] = df['bmi'].fillna(df['bmi'].mean())

In [4]:
#chuyển hóa các cộcột trong ds thành dạng [0,1]
categorical_cols = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
X = df.drop(columns=["stroke"]).values
y = df["stroke"].values
df

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67.0,0,1,1,2,1,228.69,36.600000,1,1
1,0,61.0,0,0,1,3,0,202.21,28.893237,2,1
2,1,80.0,0,1,1,2,0,105.92,32.500000,2,1
3,0,49.0,0,0,1,2,1,171.23,34.400000,3,1
4,0,79.0,1,0,1,3,0,174.12,24.000000,2,1
...,...,...,...,...,...,...,...,...,...,...,...
5105,0,80.0,1,0,1,2,1,83.75,28.893237,2,0
5106,0,81.0,0,0,1,3,1,125.20,40.000000,2,0
5107,0,35.0,0,0,1,3,0,82.99,30.600000,2,0
5108,1,51.0,0,0,1,2,0,166.29,25.600000,1,0


In [5]:
x=df.iloc[:,:10].values
x

array([[  1.        ,  67.        ,   0.        , ..., 228.69      ,
         36.6       ,   1.        ],
       [  0.        ,  61.        ,   0.        , ..., 202.21      ,
         28.89323691,   2.        ],
       [  1.        ,  80.        ,   0.        , ..., 105.92      ,
         32.5       ,   2.        ],
       ...,
       [  0.        ,  35.        ,   0.        , ...,  82.99      ,
         30.6       ,   2.        ],
       [  1.        ,  51.        ,   0.        , ..., 166.29      ,
         25.6       ,   1.        ],
       [  0.        ,  44.        ,   0.        , ...,  85.28      ,
         26.2       ,   0.        ]])

In [6]:
#chuẩn hóa dữ liệu
scaler=MinMaxScaler()
x = scaler.fit_transform(x)
x
#chuẩn hóa y
label_y=LabelEncoder()
y=label_y.fit_transform(y)
y=to_categorical(y)
y

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [8]:
# 1. Tạo mô hình
model = models.Sequential([
    layers.Input(shape=(10,)),  # 10 đặc trưng đầu vào
    layers.Dense(16, activation='relu'), #16 noron =16 lop
    layers.Dense(2, activation='softmax')  # 2 lớp đầu ra: [0, 1]
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
# Tính trọng số lớp
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(np.argmax(y_train, axis=1)),
    y=np.argmax(y_train, axis=1)
)

class_weights_dict = dict(enumerate(class_weights))

In [10]:
# Huấn luyện mô hình có trọng số lớp
model.fit(X_train, y_train,
          epochs=20, batch_size=4,
          validation_data=(X_test, y_test),
          class_weight=class_weights_dict)

Epoch 1/20
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8106 - loss: 0.6739 - val_accuracy: 0.5636 - val_loss: 0.6394
Epoch 2/20
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6880 - loss: 0.5395 - val_accuracy: 0.5646 - val_loss: 0.6426
Epoch 3/20
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6202 - loss: 0.5681 - val_accuracy: 0.6751 - val_loss: 0.5552
Epoch 4/20
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6672 - loss: 0.5579 - val_accuracy: 0.6585 - val_loss: 0.5784
Epoch 5/20
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6709 - loss: 0.5428 - val_accuracy: 0.7378 - val_loss: 0.4872
Epoch 6/20
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7125 - loss: 0.5002 - val_accuracy: 0.6018 - val_loss: 0.6526
Epoch 7/20
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7071 - loss: 0.4866 - val_accuracy: 0.6937 - val_loss: 0.5290
Epoch 8/20
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7234 - loss: 0.4835 - 

In [39]:
"""
def nhap_du_lieu():
    print("Nhập thông tin bệnh nhân để dự đoán nguy cơ đột quỵ:")
    
    gender = int(input("Giới tính (0: Nữ, 1: Nam): "))
    age = float(input("Tuổi: "))
    hypertension = int(input("Tăng huyết áp (0: Không, 1: Có): "))
    heart_disease = int(input("Bệnh tim (0: Không, 1: Có): "))
    ever_married = int(input("Đã kết hôn? (0: Chưa, 1: Rồi): "))
    work_type = int(input("Loại công việc (0: Trẻ em, 1: Chính phủ, 2: Riêng tư, 3: Tự kinh doanh, 4: Không rõ): "))
    Residence_type = int(input("Nơi sống (0: Nông thôn, 1: Thành thị): "))
    avg_glucose_level = float(input("Mức đường huyết trung bình: "))
    bmi = float(input("Chỉ số BMI: "))
    smoking_status = int(input("Hút thuốc (0: Chưa từng, 1: Thỉnh thoảng, 2: Hiện tại, 3: Không rõ): "))

    return np.array([[gender, age, hypertension, heart_disease, ever_married,
                      work_type, Residence_type, avg_glucose_level, bmi, smoking_status]])
dotquy = nhap_du_lieu()
print("Dữ liệu đầu vào:", dotquy)
"""

# Mẫu dữ liệu mới: 10 đặc trưng
dotquy = np.array([[0	,27.0,	0,	1,	1,	2,	1,	228.69,	25.600000,	1	]])
# Chuẩn hóa theo scaler đã huấn luyện
dotquy = scaler.transform(dotquy)
# Dự đoán
y_dotquy = model.predict(dotquy)
y_dotquy_pre = np.argmax(y_dotquy, axis=1)
print(y_dotquy_pre)
# In kết quả
print(f"Dự đoán: {y_dotquy_pre[0]} (0 = không đột quỵ, 1 = có đột quỵ)")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
[0]
Dự đoán: 0 (0 = không đột quỵ, 1 = có đột quỵ)


In [12]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Độ chính xác của mô hình: {accuracy:.2f}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7260 - loss: 0.4859
Độ chính xác của mô hình: 0.73


In [13]:
kq_nhan = y_dotquy_pre[0]*accuracy

In [14]:
data = pd.DataFrame({
    'nguy co': [kq_nhan]
})
print(kq_nhan)

0.0


In [15]:
def issue_warning(kq_nhan):
    if kq_nhan >= 0.8:
        return " Nguy cơ RẤT CAO - Yêu cầu kiểm tra y tế khẩn cấp!"
    elif kq_nhan >= 0.6:
        return " Nguy cơ CAO - Nên khám định kỳ và thay đổi lối sống ngay."
    elif kq_nhan >= 0.4:
        return " Nguy cơ TRUNG BÌNH - Theo dõi sức khỏe thường xuyên."
    else:
        return " Nguy cơ THẤP - Duy trì lối sống lành mạnh."


In [16]:
data['cảnh_báo'] = data['nguy co'].apply(issue_warning)
print(data)

   nguy co                                     cảnh_báo
0      0.0   Nguy cơ THẤP - Duy trì lối sống lành mạnh.


In [17]:
def prevention_advice_from_array(data):
    advice = []

    gender, age, hypertension, heart_disease, ever_married, \
    work_type, residence_type, glucose, bmi, smoking_status = data[0]

    if hypertension == 1:
        advice.append(" Kiểm soát huyết áp thường xuyên.")
    if heart_disease == 1:
        advice.append("Khám tim mạch định kỳ.")
    if smoking_status == 2:  # 2 tương đương 'smokes'
        advice.append(" Ngừng hút thuốc càng sớm càng tốt.")
    if work_type == 3:  # 3 tương đương 'Private'
        advice.append(" Nên nghỉ ngơi, tránh stress trong công việc.")
    if bmi >= 30:
        advice.append(" Giảm cân để giảm nguy cơ đột quỵ.")
    if glucose >= 140:
        advice.append(" Kiểm tra đường huyết, chế độ ăn phù hợp.")

    return " | ".join(advice) if advice else "Không có khuyến cáo đặc biệt."



In [18]:
# In kết quả
print(f"Dự đoán: {y_dotquy_pre[0]} (0 = không đột quỵ, 1 = có đột quỵ)")
print(data)

result = prevention_advice_from_array(dotquy)
print(" Khuyến cáo phòng ngừa:", result)


Dự đoán: 0 (0 = không đột quỵ, 1 = có đột quỵ)
   nguy co                                     cảnh_báo
0      0.0   Nguy cơ THẤP - Duy trì lối sống lành mạnh.
 Khuyến cáo phòng ngừa: Khám tim mạch định kỳ.
